In [ ]:
from zipfile import ZipFile
file_name="/content/drive/MyDrive/Dataset_Wild_Animals.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall("Wild_animals_Dataset")
  print('Done')

Done


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
train_path='/content/Wild_animals_Dataset/training'
test_path='/content/Wild_animals_Dataset/validation'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [ ]:
#categories
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['Cheetah', 'Hyena', 'Tiger']


In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=3):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        self.drop = nn.Dropout2d(p=0.3)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output
            

In [ ]:
model=ConvNet(num_classes=3).to(device)

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [ ]:
print(train_count,test_count)

2700 300


In [ ]:
best_accuracy=0.0

for epoch in range(100):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(round(train_accuracy,2))+' Test Accuracy: '+str(round(test_accuracy,2)))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model, "Wild_animals_CNN.pkl")
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    

Epoch: 0 Train Loss: tensor(0.6560) Train Accuracy: 0.91 Test Accuracy: 0.75
Epoch: 1 Train Loss: tensor(0.3490) Train Accuracy: 0.94 Test Accuracy: 0.68
Epoch: 2 Train Loss: tensor(0.4401) Train Accuracy: 0.93 Test Accuracy: 0.76
Epoch: 3 Train Loss: tensor(0.1792) Train Accuracy: 0.97 Test Accuracy: 0.82
Epoch: 4 Train Loss: tensor(0.1475) Train Accuracy: 0.97 Test Accuracy: 0.79
Epoch: 5 Train Loss: tensor(0.1380) Train Accuracy: 0.97 Test Accuracy: 0.78
Epoch: 6 Train Loss: tensor(0.1400) Train Accuracy: 0.97 Test Accuracy: 0.78
Epoch: 7 Train Loss: tensor(0.1348) Train Accuracy: 0.97 Test Accuracy: 0.82
Epoch: 8 Train Loss: tensor(0.2813) Train Accuracy: 0.95 Test Accuracy: 0.77
Epoch: 9 Train Loss: tensor(0.2760) Train Accuracy: 0.96 Test Accuracy: 0.64
Epoch: 10 Train Loss: tensor(0.5979) Train Accuracy: 0.93 Test Accuracy: 0.69
Epoch: 11 Train Loss: tensor(0.6019) Train Accuracy: 0.93 Test Accuracy: 0.77
Epoch: 12 Train Loss: tensor(0.5432) Train Accuracy: 0.94 Test Accuracy: 0

In [ ]:
torch.save(model, "Wild_animals_CNN.pkl")
model = torch.load("/content/Wild_animals_CNN.pkl")